In [1]:
import pickle 
import numpy as np


In [12]:
X = pickle.load(open('../unmasked_cropped.pkl','rb'))
y = pickle.load(open('../labels.pkl','rb'))
arc_lengths = pickle.load(open('./arcLengths.pkl','rb'))

In [3]:
arc_lengths = np.array(arc_lengths)
y

array([0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0,
       1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0,
       0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0,

In [4]:
arc_lengths.shape

(932,)

In [5]:
X = X/255.0

In [6]:
X

array([[[[0.04705882, 0.0745098 , 0.19215686],
         [0.04705882, 0.04313725, 0.13333333],
         [0.04313725, 0.03137255, 0.11764706],
         ...,
         [0.06666667, 0.12156863, 0.28235294],
         [0.05098039, 0.11372549, 0.28235294],
         [0.03137255, 0.10588235, 0.28235294]],

        [[0.04313725, 0.06666667, 0.18039216],
         [0.04313725, 0.03921569, 0.12941176],
         [0.04705882, 0.04313725, 0.13333333],
         ...,
         [0.03529412, 0.08627451, 0.23137255],
         [0.05098039, 0.11372549, 0.27843137],
         [0.03921569, 0.11764706, 0.30196078]],

        [[0.03921569, 0.05490196, 0.16470588],
         [0.03921569, 0.04313725, 0.12941176],
         [0.02352941, 0.03529412, 0.13333333],
         ...,
         [0.01568627, 0.04705882, 0.17647059],
         [0.05098039, 0.11372549, 0.27058824],
         [0.05098039, 0.12941176, 0.31372549]],

        ...,

        [[0.12941176, 0.26666667, 0.58039216],
         [0.0627451 , 0.19215686, 0.49019608]

In [7]:
from keras.models import Sequential,Model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, concatenate, Input

In [9]:
input_img = Input(shape=(150,150,3))
x = Conv2D(64, (3,3), activation='relu')(input_img)
x = MaxPooling2D(pool_size=(2,2))(x)
x = Conv2D(64, (3,3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2,2))(x)
x = Flatten()(x)
image_model = Model(inputs=input_img, outputs=x)

input_int = Input(shape=(1,))
y = Dense(128, activation='relu')(input_int)
gal_model = Model(inputs=input_int, outputs=y)

merged = concatenate([image_model.output, gal_model.output])
merged = Dense(128, activation='relu')(merged)
merged = Dense(2, activation='softmax')(merged)

model = Model(inputs=[image_model.input, gal_model.input], outputs=merged)

In [10]:
model.compile(optimizer='adam', loss= 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [15]:
y = np.array(y)
arc_lengths = np.array(arc_lengths)
model.fit([X,arc_lengths],y, epochs=10, validation_data=([X,arc_lengths],y))

Epoch 1/10
30/30 [==============================] - 80s 2s/step - loss: 247.4182 - accuracy: 0.5731 - val_loss: 0.8780 - val_accuracy: 0.5933
Epoch 2/10
30/30 [==============================] - 44s 1s/step - loss: 0.6997 - accuracy: 0.6821 - val_loss: 0.5564 - val_accuracy: 0.7768
Epoch 3/10
30/30 [==============================] - 47s 2s/step - loss: 0.3862 - accuracy: 0.8200 - val_loss: 0.2133 - val_accuracy: 0.9270
Epoch 4/10
30/30 [==============================] - 40s 1s/step - loss: 0.3196 - accuracy: 0.8759 - val_loss: 0.1602 - val_accuracy: 0.9442
Epoch 5/10
30/30 [==============================] - 41s 1s/step - loss: 0.1298 - accuracy: 0.9551 - val_loss: 0.0834 - val_accuracy: 0.9721
Epoch 6/10
30/30 [==============================] - 42s 1s/step - loss: 0.0712 - accuracy: 0.9761 - val_loss: 0.0565 - val_accuracy: 0.9807
Epoch 7/10
30/30 [==============================] - 42s 1s/step - loss: 0.0510 - accuracy: 0.9843 - val_loss: 0.0636 - val_accuracy: 0.9785
Epoch 8/10
30/30 [

In [16]:
model.save('64x2-CNN_impure_gal.model')

INFO:tensorflow:Assets written to: 64x2-CNN_impure_gal.model\assets
